
# Exercícios — energydata_complete.csv (26–35)

> Coloque o arquivo **`energydata_complete.csv`** na **mesma pasta** deste notebook antes de rodar.
>
> Este notebook cobre:
> - Carregamento e inspeção inicial
> - Visualizações (histogramas, séries temporais)
> - Correlações e PCA
> - Modelos: Regressão Linear Múltipla, Random Forest Regressor
> - Clustering K-Means
> - Classificação binária (alto/baixo consumo) com métricas
>
> Bibliotecas usadas: **pandas, numpy, matplotlib, scikit-learn, seaborn (opcional)**, **statsmodels** (opcional).


In [ ]:

# %pip install pandas numpy matplotlib scikit-learn seaborn statsmodels


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

plt.rcParams.update({'figure.figsize': (10,4), 'axes.grid': True})

DATA_FILE = "energydata_complete.csv"
if not Path(DATA_FILE).exists():
    print(f"⚠️ Arquivo '{DATA_FILE}' não encontrado em {Path.cwd()}")
else:
    print("✅ Arquivo encontrado.")


## 26) Carregamento e inspeção inicial

In [ ]:

if Path(DATA_FILE).exists():
    df = pd.read_csv(DATA_FILE)
    print('Shape:', df.shape)
    print('\n.info():')
    display(df.info())
    print('\n.describe():')
    display(df.describe().T)
else:
    print("Coloque o arquivo na mesma pasta do notebook e rode novamente.")


## 27) Distribuição do consumo — `Appliances`

In [ ]:

if Path(DATA_FILE).exists():
    # Histograma
    plt.figure()
    df['Appliances'].hist(bins=50)
    plt.title("Histograma — Appliances")
    plt.xlabel("Appliances (Wh)")
    plt.ylabel("Frequência")
    plt.show()

    # Série temporal (amostra para visualização se for muito grande)
    plt.figure()
    # converter date se existir coluna 'date' e plotar uma janela de 7 dias
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        sample = df.set_index('date')['Appliances'].resample('1H').mean().dropna()
        sample_window = sample.sort_index().iloc[:24*7]  # primeira semana
        plt.plot(sample_window.index, sample_window.values)
        plt.title("Série temporal — Appliances (primeira semana)")
        plt.xlabel("Data")
        plt.ylabel("Appliances (Wh)")
        plt.show()
    else:
        plt.plot(df['Appliances'].values[:24*7])
        plt.title("Série temporal (primeiros 7 dias aproximados) — Appliances")
        plt.xlabel("Índice")
        plt.ylabel("Appliances (Wh)")
        plt.show()

    print("Pergunta: o consumo tende a se concentrar em valores baixos ou altos?")
    print("Observe o histograma e a escala (Wh). Em muitos datasets de consumo doméstico, a massa está em valores baixos com cauda à direita.")
else:
    print("Arquivo não encontrado.")


## 28) Correlações com variáveis ambientais

In [ ]:

if Path(DATA_FILE).exists():
    # Selecionar variáveis ambientais típicas (ajuste se nomes diferentes)
    candidates = ['T_out','T1','T2','T3','T4','T5','T6','T7','T8','T9',
                  'RH_out','RH_1','RH_2','RH_3','RH_4','RH_5','RH_6','RH_7','RH_8','RH_9']
    present = [c for c in candidates if c in df.columns]
    cols = ['Appliances'] + present
    corr = df[cols].corr()
    display(corr.loc['Appliances'].sort_values(ascending=False))
    print("\nPergunta: quais fatores têm mais relação com o consumo?")
    print("Olhe os maiores valores absolutos de correlação com 'Appliances'.")
else:
    print("Arquivo não encontrado.")


## 29) Normalização (Min-Max Scaling) das variáveis numéricas

In [ ]:

if Path(DATA_FILE).exists():
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    scaler = MinMaxScaler()
    df_scaled = df.copy()
    df_scaled[num_cols] = scaler.fit_transform(df[num_cols].fillna(0))
    print("Exemplo das colunas numéricas escaladas:")
    display(df_scaled[num_cols].head())
else:
    print("Arquivo não encontrado.")


## 30) PCA — reduzir para 2 componentes principais

In [ ]:

if Path(DATA_FILE).exists():
    numeric = df.select_dtypes(include=[np.number]).dropna(axis=1, how='all')
    cols_pca = numeric.columns.tolist()
    Xp = MinMaxScaler().fit_transform(numeric.fillna(0))
    pca = PCA(n_components=2, random_state=42)
    Z = pca.fit_transform(Xp)
    print("Explained variance ratio:", pca.explained_variance_ratio_)
    plt.figure()
    plt.scatter(Z[:,0], Z[:,1], s=4, alpha=0.6)
    plt.title("PCA — 2 componentes (todos os registros)")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.show()
    print("Pergunta: aparecem padrões ou agrupamentos naturais? Observe o gráfico para possíveis regiões de maior densidade.")
else:
    print("Arquivo não encontrado.")


## 31) Regressão Linear Múltipla — Appliances ~ variáveis ambientais

In [ ]:

if Path(DATA_FILE).exists():
    # Definir features ambientais — escolher as presentes
    env_candidates = ['T_out','RH_out']  # defaults
    # expandir se colunas T1.. existirem: escolha algumas representativas
    for c in ['T1','T2','T3','RH_1','RH_2','RH_3']:
        if c in df.columns:
            env_candidates.append(c)
    features = [c for c in env_candidates if c in df.columns]
    data = df[features + ['Appliances']].dropna()
    X = data[features].values
    y = data['Appliances'].values

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

    lr = LinearRegression()
    lr.fit(X_train,y_train)
    y_pred = lr.predict(X_test)

    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    print("Features usadas:", features)
    print("R^2:", r2)
    print("RMSE:", rmse)
else:
    print("Arquivo não encontrado.")


## 32) Random Forest Regressor — comparar com Regressão Linear

In [ ]:

if Path(DATA_FILE).exists():
    # Reusar X_train, X_test, y_train, y_test do passo anterior (se não existirem, refazer)
    try:
        X_train, X_test, y_train, y_test
    except NameError:
        data = df[features + ['Appliances']].dropna()
        X = data[features].values
        y = data['Appliances'].values
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train,y_train)
    y_pred_rf = rf.predict(X_test)
    rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
    print("RMSE Regressão Linear (ref):", rmse)
    print("RMSE Random Forest:", rmse_rf)
else:
    print("Arquivo não encontrado.")


## 33) K-Means clustering (k=3..5)

In [ ]:

if Path(DATA_FILE).exists():
    # Usar apenas variáveis numéricas escaladas para clustering
    num = df.select_dtypes(include=[np.number]).fillna(0)
    num_s = MinMaxScaler().fit_transform(num)
    inertias = {}
    for k in range(3,6):
        km = KMeans(n_clusters=k, random_state=42, n_init=10)
        km.fit(num_s)
        inertias[k] = km.inertia_
        print(f"k={k}, inertia={km.inertia_:.2f}")
    # Aplicar k=3 por exemplo e mostrar centros (desescalados apenas para interpretação relativa)
    km3 = KMeans(n_clusters=3, random_state=42, n_init=10)
    labels = km3.fit_predict(num_s)
    df['k3_cluster'] = labels
    print("Tamanho dos clusters (k=3):", df['k3_cluster'].value_counts().to_dict())
    display(df.groupby('k3_cluster')['Appliances'].describe())
    print("Interprete os perfis por média/mediana de 'Appliances' e outras variáveis.")
else:
    print("Arquivo não encontrado.")


## 34) Classificação binária — alto vs baixo consumo

In [ ]:

if Path(DATA_FILE).exists():
    median = df['Appliances'].median()
    df['high_consumption'] = (df['Appliances'] > median).astype(int)
    print("Mediana Appliances:", median)
    display(df['high_consumption'].value_counts())

    # Selecionar features — usar as mesmas ambientais do modelo de regressão
    data_cls = df[features + ['high_consumption']].dropna()
    Xc = data_cls[features].values
    yc = data_cls['high_consumption'].values
    Xc_train,Xc_test,yc_train,yc_test = train_test_split(Xc,yc,test_size=0.2,random_state=42)
    # Logistic Regression
    log = LogisticRegression(max_iter=1000)
    log.fit(Xc_train,yc_train)
    y_pred_log = log.predict(Xc_test)
    # Random Forest Classifier
    rfc = RandomForestClassifier(n_estimators=100, random_state=42)
    rfc.fit(Xc_train,yc_train)
    y_pred_rfc = rfc.predict(Xc_test)

    print("Modelos treinados: Logistic Regression e Random Forest (Classifier)")
else:
    print("Arquivo não encontrado.")


## 35) Avaliação de classificação

In [ ]:

if Path(DATA_FILE).exists():
    print("=== Logistic Regression ===")
    print("Accuracy:", accuracy_score(yc_test, y_pred_log))
    print("Precision:", precision_score(yc_test, y_pred_log))
    print("Recall:", recall_score(yc_test, y_pred_log))
    print("F1:", f1_score(yc_test, y_pred_log))
    print("\nConfusion matrix:")
    print(confusion_matrix(yc_test, y_pred_log))

    print("\n=== Random Forest Classifier ===")
    print("Accuracy:", accuracy_score(yc_test, y_pred_rfc))
    print("Precision:", precision_score(yc_test, y_pred_rfc))
    print("Recall:", recall_score(yc_test, y_pred_rfc))
    print("F1:", f1_score(yc_test, y_pred_rfc))
    print("\nConfusion matrix:")
    print(confusion_matrix(yc_test, y_pred_rfc))

    print("\nPergunta: o modelo erra mais para alto ou para baixo consumo?")
    print("Compare valores de falso positivo (prevê alto quando é baixo) e falso negativo (prevê baixo quando é alto) nas matrizes de confusão.")
else:
    print("Arquivo não encontrado.")
